# Overview

This notebook runs the code for performing schooling experiments. We used a Luxli ORC-Taiko-2x1 LED light source controlled with an Enttex DMX USB Pro via a 5-pin DMX cable. The Python package to control this hardware (DMXEnttecPro) can be found on [github](https://github.com/SavinaRoja/DMXEnttecPro).

# Packages

In [1]:
from DMXEnttecPro import Controller
import numpy as np
import def_runexperiments as re

# Hardware setup

You can first find the Enttec device by typing the folling at the terminal:

> python -m DMXEnttecPro.utils

which gave me the following:

>/dev/cu.usbserial-EN373474 <br>
>  name: cu.usbserial-EN373474 <br> 
>  description: DMX USB PRO<br>
>  hwid: USB VID:PID=0403:6001 SER=EN373474 LOCATION=0-1.1.1<br>
>  vid: 1027<br>
>  pid: 24577<br>
>  serial_number: EN373474<br>
>  location: 0-1.1.1<br>
>  manufacturer: ENTTEC<br>
>  product: DMX USB PRO<br>
>  interface: None<br>

The first line is what you'll want to copy and paste below for your hardware.

## Parameters

In [2]:
# Address for the Enttec USB device (see above)
hw_address = "/dev/cu.usbserial-EN373474"

# Path to time-code audio file
aud_path = "/Users/mmchenry/Documents/Projects/waketracking/test.m4a"

## Initialize hardware

Run these lines for all of the programs that you want to run below.

In [3]:
#  Initialize hardware
dmx = Controller(hw_address, auto_submit=True, dmx_size=32)

# Set output rate at maximum
dmx.set_dmx_parameters(output_rate=0)

# Programs

Select which of the cells below that make sense for the experiment that you want to run.

## Fixed light intensity

In [4]:
# Specify light intensity (from 0 to 1)
light_intensity = 0.5

# Sets DMX channel 1 to max 255 (Channel 1 is the intensity)
dmx.set_channel(1, int(light_intensity*255))  

## Ramp down and then up

Changing light intensity at a constant rate.

In [4]:

# list of light levels
light_level = np.array([1, 0, 0.5])

# Duration that each level is fixed
light_dur = np.array([2, 3, 4])

# Duration for the changes in light level
ramp_dur = np.array([0.5, 0.1])

# Dataframe of light and control levels 
df = re.make_ramp(light_level, light_dur, ramp_dur, plot_data=False)

# Play control levels into Enttec DMX
re.run_program(df, dmx, aud_path=aud_path)

# print(df)

Time (s): 92.49, Writing to channel 1: 0.49
